In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
currentMVPs = pd.read_csv('Data/currentMVPRace.csv')
pastMVPS = pd.read_csv('Data/pastMVPS.csv')

In [3]:
train, test = train_test_split(pastMVPS, test_size=0.2, random_state=5)

Xtrain = train[['G', 'Team Wins', 'Overall Seed', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48', 'VORP', 'BPM']]
Ytrain = train[['Share']]
Xtest = test[['G', 'Team Wins', 'Overall Seed', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48', 'VORP', 'BPM']]
Ytest = test[['Share']]